## Memory in Langgraph

In [1]:
import os
from typing import TypedDict, Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")


In [3]:
llm = ChatOllama(model="glm-5:cloud", base_url=OLLAMA_BASE_URL)

# checking the response
result = llm.invoke("Hello!, how are you?")
print(result)

content="Hello! I'm doing well, thank you for asking. As an AI, I don't have feelings, but I'm functioning perfectly and ready to help you.\n\nHow are you doing today? Is there something specific I can help you with?" additional_kwargs={} response_metadata={'model': 'glm-5:cloud', 'created_at': '2026-02-23T05:49:27.716999524Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6241148412, 'load_duration': None, 'prompt_eval_count': 11, 'prompt_eval_duration': None, 'eval_count': 491, 'eval_duration': None, 'logprobs': None, 'model_name': 'glm-5:cloud', 'model_provider': 'ollama'} id='lc_run--019c890b-8bfc-7191-bccb-e648328c492a-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 11, 'output_tokens': 491, 'total_tokens': 502}


In [6]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [7]:
from langchain_core.tools import tool
from datetime import datetime

@tool
def date_time_tool() -> str:
    """Returns the current date and time."""
    return datetime.now().isoformat()

In [9]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import BaseMessage

llm_with_tools = llm.bind_tools([date_time_tool])

# Update the chatbot function to use llm_with_tools
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tool_node = ToolNode(tools=[date_time_tool])

# Rebuild the graph with the new tools
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")
graph = graph_builder.compile()

In [10]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

**Explanation:**

`MemorySaver` is a checkpointer that stores the graph state in memory:

- **`MemorySaver()`** - Creates an in-memory checkpointer to persist conversation state
- **`checkpointer=memory`** - Passed to `compile()` to enable state persistence across invocations
- Each conversation thread is identified by a `thread_id` in the config

This allows the chatbot to remember previous messages in the conversation.

In [11]:
config = {"configurable": {"thread_id": "1"}}

**Explanation:**

The `config` dictionary configures the conversation thread:

- **`thread_id`** - A unique identifier for the conversation session
- Different `thread_id` values create separate conversation histories
- Same `thread_id` continues the existing conversation with memory

**Example:**
- `thread_id: "1"` → One conversation history
- `thread_id: "2"` → A completely separate conversation

In [12]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # Process user input through the LangGraph
    for event in graph.stream({"messages": [("user", user_input)]}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: Hi there! How can I help you today?
Assistant: 
Assistant: 2026-02-23T11:22:41.856339
Assistant: Today's date is February 23, 2026. The current time is 11:22 AM.
Assistant: I can help you with a wide variety of tasks! Here are some of the things I can do:

**Writing & Communication**
- Help write, edit, or proofread documents, emails, essays, and more
- Assist with creative writing, stories, or poems
- Help improve your writing style and clarity

**Research & Information**
- Answer questions on countless topics
- Explain complex concepts in simple terms
- Help you learn about new subjects

**Problem Solving**
- Help with math, coding, and logic problems
- Brainstorm ideas and solutions
- Break down complex problems into manageable steps

**Productivity**
- Help organize tasks and priorities
- Create plans, outlines, and summaries
- Assist with study guides and learning materials

**Creative Projects**
- Help with brainstorming and ideation
- Provide feedback and suggestions


**Note on `MemorySaver`:**

`MemorySaver` is an **in-memory** checkpointer, which means:

- **Volatile Storage**: State is lost when the Python process restarts
- **Best for**: Development, testing, and short-lived conversations
- **Not for Production**: Use persistent checkpointers for production:
  - `SqliteSaver` - SQLite database persistence
  - `PostgresSaver` - PostgreSQL database persistence
  - `RedisSaver` - Redis-based persistence

**How Memory Works:**
1. Each `graph.stream()` or `graph.invoke()` call with the same `thread_id` continues from the previous state
2. The checkpointer saves the state after each node execution
3. On the next invocation, the graph loads the previous state and appends new messages